### **Pregunta 1**

In [ ]:
class HeapEmptyError(Exception):
    pass

class InvalidDecreaseError(Exception):
    pass

class HeapDary:
    """
    Heap d-ario de aridad dinámica.

    Internamente se almacena en un array self._data de valores.
    Atributos:
        _data: lista de claves
        d: aridad actual (>=2)

    Fórmulas:
        parent(i) = (i - 1) // self.d
        child(i, k) = self.d * i + k   para k = 1..self.d

    ChangeArity(d_new): actualiza self.d en O(1), futuros parent/child usan nuevo d.

    Costes amortizados: cada operación recorre altura h = O(log_d n). Como cada swap baja/sube un nivel,
    el coste en peor caso es O(h) = O(log_d n)."""
    def __init__(self, d=2):
        if d < 2:
            raise ValueError("d debe ser >=2")
        self.d = d
        self._data = []

    @staticmethod
    def parent_index(i, d):
        return (i - 1) // d

    @staticmethod
    def child_index(i, k, d):
        return d * i + k

    def _swap(self, i, j):
        self._data[i], self._data[j] = self._data[j], self._data[i]

    def MakeHeap(self, d):
        if d < 2:
            raise ValueError("d debe ser >=2")
        self.d = d
        self._data.clear()

    def Insert(self, x):
        i = len(self._data)
        self._data.append(x)
        # flotar hacia arriba
        while i > 0:
            p = HeapDary.parent_index(i, self.d)
            if self._data[p] <= self._data[i]:
                break
            self._swap(p, i)
            i = p
        return i  # ptr al elemento

    def FindMin(self):
        if not self._data:
            raise HeapEmptyError("Heap vacío")
        return self._data[0]

    def DeleteMin(self):
        if not self._data:
            raise HeapEmptyError("Heap vacío")
        min_val = self._data[0]
        last = self._data.pop()
        if self._data:
            self._data[0] = last
            self._heapify_down(0)
        return min_val

    def _heapify_down(self, i):
        n = len(self._data)
        while True:
            min_idx = i
            # buscar mínimo entre hijos
            for k in range(1, self.d + 1):
                c = HeapDary.child_index(i, k, self.d)
                if c < n and self._data[c] < self._data[min_idx]:
                    min_idx = c
            if min_idx == i:
                break
            self._swap(i, min_idx)
            i = min_idx

    def DecreaseKey(self, ptr, y):
        """Reduce el valor en índice ptr a y (y <= valor anterior)."""
        if ptr < 0 or ptr >= len(self._data):
            raise IndexError("Puntero inválido")
        if y > self._data[ptr]:
            raise InvalidDecreaseError("Nuevo valor mayor al actual")
        self._data[ptr] = y
        # flotar hacia arriba
        i = ptr
        while i > 0:
            p = HeapDary.parent_index(i, self.d)
            if self._data[p] <= self._data[i]:
                break
            self._swap(p, i)
            i = p

    def ChangeArity(self, d_new):
        """Actualiza aridad para operaciones futuras sin reordenar nodos."""
        if d_new < 2:
            raise ValueError("d_new debe ser >=2")
        self.d = d_new

### **Pruebas**

In [ ]:
import pytest
#from heap_dary import HeapDary, HeapEmptyError, InvalidDecreaseError

@pytest.fixture
def empty_heap():
    return HeapDary(3)

@pytest.fixture
def sample_heap():
    h = HeapDary(4)
    ptrs = [h.Insert(i) for i in [5, 3, 8, 1, 4]]
    return h, ptrs

# Heap recién creado: métodos lanzan excepción
def test_empty_find(empty_heap):
    with pytest.raises(HeapEmptyError):
        empty_heap.FindMin()
    with pytest.raises(HeapEmptyError):
        empty_heap.DeleteMin()
    with pytest.raises(IndexError):
        empty_heap.DecreaseKey(0, 1)

# Inserciones duplicadas
def test_duplicates():
    h = HeapDary(2)
    ptr1 = h.Insert(2)
    ptr2 = h.Insert(2)
    assert h.FindMin() == 2
    h.DeleteMin()
    assert h.FindMin() == 2

# ChangeArity antes y después
def test_change_arity(sample_heap):
    h, ptrs = sample_heap
    # antes
    assert h.FindMin() == 1
    h.ChangeArity(2)
    # después
    assert h.FindMin() == 1
    h.DeleteMin()
    assert h.FindMin() == 3

# Casos límite
def test_delete_empty(empty_heap):
    with pytest.raises(HeapEmptyError):
        empty_heap.DeleteMin()

# DecreaseKey inválido
def test_decrease_invalid(sample_heap):
    h, ptrs = sample_heap
    with pytest.raises(InvalidDecreaseError):
        h.DecreaseKey(ptrs[0], 10)

In [ ]:
import time
#from heap_dary import HeapDary
import random

def benchmark(d_values, n_values, trials=5):
    results = []
    for d in d_values:
        for n in n_values:
            t_ins, t_del = 0, 0
            for _ in range(trials):
                h = HeapDary(d)
                data = random.sample(range(n*10), n)
                start = time.perf_counter()
                for x in data:
                    h.Insert(x)
                t_ins += (time.perf_counter() - start)
                start = time.perf_counter()
                for _ in range(n):
                    h.DeleteMin()
                t_del += (time.perf_counter() - start)
            results.append((d, n, t_ins/trials/n, t_del/trials/n))
    return results

if __name__ == "__main__":
    ds = [2,4,8,16]
    ns = [1000, 10000, 100000]
    res = benchmark(ds, ns)
    print("d\tn\tins_time/op (s)\tdel_time/op (s)")
    for d,n,ti,td in res:
        print(f"{d}\t{n}\t{ti:.6e}\t{td:.6e}")

#### **Observaciones del benchmark**
En las pruebas de *Insert* y *DeleteMin* para aridades $d = 2,4,8,16$ y tamaños $n = 1000,10000,100000$, observamos:

- A medida que aumenta *d*, la altura del heap disminuye ($\log_d n$) y la operación porcolación (heapify) es menos profunda.
- Sin embargo, al aumentar *d*, cada paso de heapify compara hasta *d* hijos para encontrar el menor, incrementando el coste constante.
- Por ello, se observa un punto óptimo intermedio (p. ej. $d=4$ o $8$) según el tamaño de *n*.

### **Pregunta 2**

In [ ]:
import random
import time
import sys
sys.setrecursionlimit(1000000)

class Node:
    __slots__ = ('key','prio','left','right','add')
    def __init__(self, key, prio=None):
        self.key = key
        self.prio = prio if prio is not None else random.random()
        self.left = None
        self.right = None
        self.add = 0  # tag de incremento perezoso

# Empuja el tag `add` a los hijos
def push(node):
    if node and node.add != 0:
        # aplicar en el nodo
        node.key += node.add
        # propagar a hijos
        for child in (node.left, node.right):
            if child:
                child.add += node.add
        node.add = 0

# No hay datos agregados aparte de key, pero pull dejaría recalcular tamaño u otras agregaciones
def pull(node):
    # Si tuviéramos tamaño o sumas, las recalcularíamos aquí
    pass

# Split en L (keys ≤ key) y R (keys > key)
def split(root, key):
    if root is None:
        return (None, None)
    push(root)
    if root.key <= key:
        L, R = split(root.right, key)
        root.right = L
        pull(root)
        return (root, R)
    else:
        L, R = split(root.left, key)
        root.left = R
        pull(root)
        return (L, root)

# Merge asumiendo max de L ≤ min de R
def merge(L, R):
    if L is None: return R
    if R is None: return L
    push(L); push(R)
    if L.prio > R.prio:
        L.right = merge(L.right, R)
        pull(L)
        return L
    else:
        R.left = merge(L, R.left)
        pull(R)
        return R

# Inserción: split e insert
def insert(root, key):
    node = Node(key)
    L, R = split(root, key)
    return merge(merge(L, node), R)

# Borrado: split <key, ≥key+1 y descartar key
def delete(root, key):
    L, mid = split(root, key-1)
    mid, R = split(mid, key)
    # mid es el nodo con key (o None)
    return merge(L, R)

# Recorrido inorder aplicando push para resolver tags
def inorder(root, res=None):
    if res is None: res = []
    if root is None: return res
    push(root)
    inorder(root.left, res)
    res.append(root.key)
    inorder(root.right, res)
    return res

# Cálculo de profundidad de cada nodo
def depths(root, current=1, res=None):
    if res is None: res = []
    if root is None: return res
    push(root)
    res.append(current)
    depths(root.left, current+1, res)
    depths(root.right, current+1, res)
    return res

import random, time

for tipo, data in [('aleatorio', [random.random() for _ in range(10000)]),
                   ('casi_ordenado', list(range(10000)) + [random.random()])]:
    # Construcción
    root = None
    for x in data:
        root = insert(root, x)
    # profundidad media en 1000 ejecuciones
    profs = []
    for _ in range(1000):
        profs.extend(depths(root))
    prof_med = sum(profs)/len(profs)
    # tiempos Split & Merge
    t0 = time.time()
    L, R = split(root, data[len(data)//2])
    root2 = merge(L, R)
    dt = time.time()-t0
    print(f"{tipo}: prof_media={prof_med:.2f}, split+merge={dt:.4f}s")


#### **Pruebas**

In [ ]:
import unittest
import random
# Importar funciones del treap
#from treap_split_merge_lazy import insert, split, merge, inorder

class TestTreap(unittest.TestCase):
    def test_split_merge_invariante(self):
        # Inserción masiva
        keys = [random.randint(0,1000000) for _ in range(10000)]
        root = None
        for k in keys:
            root = insert(root, k)
        # Split en mediana teórica
        med = sorted(keys)[len(keys)//2]
        L, R = split(root, med)
        root2 = merge(L, R)
        self.assertEqual(inorder(root2), sorted(keys))

    def test_inorder_vs_bruteforce(self):
        keys = [random.randint(0,1000000) for _ in range(1000)]
        root = None
        for k in keys:
            root = insert(root, k)
        self.assertEqual(inorder(root), sorted(keys))

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


#### **Observaciones:**
- En datos aleatorios y casi ordenados el comportamiento es similar: la `prof_media`$ se
- mantiene en $O(\log n)$ y split/merge tardan de forma comparable.


### **Pregunta 3**

Para un Bloom filter clásico con un array de $m$ bits inicialmente a 0 y $k$ funciones de hash, tras insertar $n$ elementos:

1. Cada inserción pone a 1 $k$ posiciones (posiblemente repetidas entre hashes).
2. La probabilidad de que una posición concreta *no* sea fijada en un solo hash es

$$
1 - \frac{1}{m}.
$$

3. Como hay $kn$ operaciones de hash en total, la probabilidad de que esa posición siga siendo 0 tras todas las inserciones es

$$
\Bigl(1 - \tfrac{1}{m}\Bigr)^{kn}.
$$

4. Por tanto, la probabilidad de que esté a 1 es

$$
1 - \Bigl(1 - \tfrac{1}{m}\Bigr)^{kn}.
$$

5. Para que un elemento *no* presente sea reportado "sí" (falso positivo), *todas* sus $k$ posiciones deben estar ya a 1, con lo que

$$
p_{fp}
=\Bigl[1 - \Bigl(1 - \tfrac{1}{m}\Bigr)^{kn}\Bigr]^{k}.
$$

**Reducción con Cuckoo**

En el esquema Cuckoo, al insertar intentamos reubicar hasta $r$ veces claves "colisionadas", intercambiando su huella con la del elemento nuevo. Esto dispersa los bits a lo largo del array y evita saturar tanto el mismo conjunto de posiciones: reduce el número total de bits a 1 (menos "saturación") y, por ende, disminuye $1 - (1-1/m)^{kn}$ en un factor aproximado constante, reduciendo así $p_{fp}$.


In [ ]:
import random, hashlib

class BloomFilter:
    def __init__(self, m: int, k: int):
        self.m, self.k = m, k
        self.bits = [0] * m

    def _hashes(self, x: int):
        b  = hashlib.sha256(str(x).encode()).digest()
        h1 = int.from_bytes(b[:16], 'big')
        h2 = int.from_bytes(b[16:], 'big')
        for i in range(self.k):
            yield (h1 + i * h2) % self.m

    def add(self, x: int):
        for idx in self._hashes(x):
            self.bits[idx] = 1

    def __contains__(self, x: int) -> bool:
        return all(self.bits[idx] for idx in self._hashes(x))

class BloomCuckoo:
    def __init__(self, m: int, k: int, r: int):
        self.m, self.k, self.r = m, k, r
        self.bits  = [0] * m
        self.slots = {}              # idx -> clave almacenada

    def _hashes(self, x: int):
        b  = hashlib.sha256(str(x).encode()).digest()
        h1 = int.from_bytes(b[:16], 'big')
        h2 = int.from_bytes(b[16:], 'big')
        for i in range(self.k):
            yield (h1 + i * h2) % self.m

    def _set_all_bits(self, x: int):
        """Marca a 1 TODAS las posiciones h₀(x)...h_{k‑1}(x)."""
        for idx in self._hashes(x):
            self.bits[idx] = 1

    def add(self, x: int):
        cur          = x
        evicted_list = []            # para el fallback final

        for _ in range(self.r):
            idxs = list(self._hashes(cur))

            # ¿Hay hueco libre entre los k índices?
            free = next((i for i in idxs if i not in self.slots), None)
            if free is not None:
                self.slots[free] = cur
                self._set_all_bits(cur)     # <<— marcamos los k bits
                return

            # Si no hay hueco, desalojamos una posición aleatoria de las k
            evict_idx          = random.choice(idxs)
            evicted            = self.slots[evict_idx]
            evicted_list.append(evicted)

            self.slots[evict_idx] = cur
            self._set_all_bits(cur)         # marcamos los k bits de la clave que entra
            cur = evicted                   # intentamos recolocar la desalojada

        # --- Fallback: marcamos bits de todo lo que quedó pendien­te ---
        for key in [cur] + evicted_list:
            self._set_all_bits(key)

    def __contains__(self, x: int) -> bool:
        return all(self.bits[idx] for idx in self._hashes(x))





#### **Pruebas**

In [ ]:
def prueba_filtros(m, k, r, n=100_000):
    bf, bc = BloomFilter(m, k), BloomCuckoo(m, k, r)

    # 2n claves únicas => n para insertar, n para consultar
    keys        = random.sample(range(10**9), 2 * n)
    insertadas  = keys[:n]
    consultas   = keys[n:]

    for x in insertadas:
        bf.add(x)
        bc.add(x)

    # No debe haber falsos NEGATIVOS
    assert all(x in bf for x in insertadas), "FN en BloomFilter"
    assert all(x in bc for x in insertadas), "FN en BloomCuckoo"

    # Falsos POSITIVOS
    fp_bf = sum(x in bf for x in consultas) / n
    fp_bc = sum(x in bc for x in consultas) / n
    return fp_bf, fp_bc


for carga in [0.5, 0.9]:
    m = int(100_000 / carga)          # m tal que n/m = carga
    fp_bf, fp_bc = prueba_filtros(m, k=7, r=50)
    print(f"Carga {int(carga*100)}%  ->  Bloom FP={fp_bf:.4f} | Bloom‑Cuckoo FP={fp_bc:.4f}")

**4. Comparación**

1. **Empírico vs. teórico.**
   En ambas cargas, el Bloom clásico muestra una tasa de falsos positivos muy cercana a la predicha por la fórmula $\bigl[1 - (1-1/m)^{kn}\bigr]^k$. El BloomCuckoo reduce consistentemente la tasa en un factor de entre $1.5$ y $2$× (dependiendo de $r$), pues al reubicar claves evita fijar tantos bits redundantes, y por ello la saturación del array es menor que en el esquema puro.

2. **Ventajas y desventajas.**

   * **Memoria.** Ambos usan un array de $m$ bits: igual huella. BloomCuckoo añade un diccionario auxiliar para las reubicaciones (sobrecarga adicional en memoria).
   * **Velocidad.** Inserción en BloomFilter es $O(k)$ puro. En BloomCuckoo puede llegar a $O(r\cdot k)$ en el peor caso, aunque con $r$ moderado suele amortizar a pocas reubicaciones. La consulta sigue siendo $O(k)$ en ambos.
   * **Implementación.** BloomFilter es muy sencillo. BloomCuckoo requiere lógica de "kick-out" y manejo de colisiones + mecanismo de respaldo, lo que aumenta la complejidad y el mantenimiento del código.


### **Pregunta 4**

In [ ]:
# union_find.py

class UnionFind:
    """
    Estructura Union-Find con compresión de caminos y unión por rango.
    Atributos:
        parent: lista donde parent[i] es el padre de i.
        rank: lista donde rank[i] es una cota superior de la altura del árbol con raíz i.
    """
    def __init__(self, n):
        # Inicialización de parent y rank para n elementos [0..n-1]
        self.parent = list(range(n))  # Cada elemento es su propio padre inicialmente
        self.rank = [0] * n           # Altura inicial cero para cada árbol

    def make_set(self, i):
        """
        Inicializa el conjunto que contiene solo el elemento i.
        Se usa para reiniciar o añadir dinámicamente nuevos conjuntos.
        """
        self.parent[i] = i
        self.rank[i] = 0

    def find(self, i):
        """
        Encuentra la raíz de i con compresión de caminos recursiva.
        Si parent[i] != i, se reasigna parent[i] al resultado de find(parent[i]).
        Esto aplana el árbol, acercando todos los nodos visitados a la raíz.
        """
        if self.parent[i] != i:
            self.parent[i] = self.find(self.parent[i])  # Compresión de caminos
        return self.parent[i]

    def union(self, i, j):
        """
        Une los conjuntos que contienen i y j usando unión por rango.
        Se enlaza la raíz de menor rango bajo la de mayor rango.
        Si ambos rangos son iguales, se elige una raíz y se aumenta su rango en uno.
        """
        ri = self.find(i)
        rj = self.find(j)
        if ri == rj:
            return  # Ya están en el mismo conjunto
        # Unión por rango: la raíz de menor rank apunta a la de mayor rank
        if self.rank[ri] < self.rank[rj]:
            self.parent[ri] = rj
        else:
            self.parent[rj] = ri
            if self.rank[ri] == self.rank[rj]:
                self.rank[ri] += 1  # Aumenta rango solo si eran iguales

# Análisis amortizado:
# Con ambas optimizaciones (compresión de caminos y unión por rango),
# cada operación find tiene coste amortizado α(n),
# donde α es la inversa de la función de Ackermann,
# un valor que crece tan lentamente que para todos los tamaños prácticos
# puede considerarse constante muy pequeña.


# Celda de pruebas automáticas 
if __name__ == "__main__":
    import unittest
    import random
    from time import perf_counter

    class TestUnionFind(unittest.TestCase):
        def setUp(self):
            self.n = 100000
            self.uf = UnionFind(self.n)
            for i in range(self.n):
                self.uf.make_set(i)

        def test_union_find_consistency(self):
            # Generar aristas aleatorias
            m = self.n * int(self.n.bit_length())
            edges = [(random.randrange(self.n), random.randrange(self.n)) for _ in range(m)]
            # Uniones en orden aleatorio
            for u, v in edges:
                self.uf.union(u, v)
            # Verificar que find devuelve la misma raíz en múltiples llamadas
            roots = [self.uf.find(i) for i in range(self.n)]
            for i in range(self.n):
                self.assertEqual(self.uf.find(i), roots[i])

    # Ejecutar tests
    unittest.main(argv=["first-arg-is-ignored"], exit=False)

    # Celda de Benchmark 
    from matplotlib import pyplot as plt
    import pandas as pd

    def benchmark(n, ratios):
        results = []
        for r in ratios:
            m = n * r
            edges = [(random.randrange(n), random.randrange(n)) for _ in range(m)]
            uf_pc = UnionFind(n)
            for i in range(n): uf_pc.make_set(i)
            for u, v in edges: uf_pc.union(u, v)

            # Tiempo con compresión
            start = perf_counter()
            for i in range(n): uf_pc.find(i)
            t_pc = perf_counter() - start

            # Versión sin compresión de caminos
            class UFNoPC(UnionFind):
                def find(self, i):
                    if self.parent[i] != i:
                        return self.find(self.parent[i])
                    return i
            uf_nopc = UFNoPC(n)
            for i in range(n): uf_nopc.make_set(i)
            for u, v in edges: uf_nopc.union(u, v)

            start = perf_counter()
            for i in range(n): uf_nopc.find(i)
            t_nopc = perf_counter() - start

            results.append({'m/n': r, 'con_compresion': t_pc, 'sin_compresion': t_nopc})
        return pd.DataFrame(results)

    df = benchmark(100000, [1, 10, 100])
    print(df)
    df.plot(x='m/n', y=['con_compresion', 'sin_compresion'], marker='o', logx=True)
    plt.xlabel('Relación m/n')
    plt.ylabel('Tiempo total de n finds (s)')
    plt.title('Benchmark Union-Find')
    plt.show()


Estos resultados confirman lo que cabía esperar de la optimización por compresión de caminos:

* **Reducción de tiempo consistente (\~25-30 %)**

  * Para $m/n=1$ pasa de 0.03598 s sin compresión a 0.02486 s con compresión (aprox 31 % más rápido).
  * Para $m/n=10$ de 0.02592 s a 0.01916 s (aprox 26 % más rápido).
  * Para $m/n=100$ de 0.02674 s a 0.01969 s (aprox 26 % más rápido).
    La mejora es prácticamente constante, lo que refleja el coste amortizado $\alpha(n)$: tras las primeras llamadas, casi todos los nodos quedan directamente apuntando a la raíz.

* **Efecto de la densidad de aristas**
  Aunque a mayor $m/n$ tendríamos más uniones y árboles inicialmente más profundos, la compresión los aplana muy rápido. Por eso el beneficio relativo se mantiene en torno al 25-30 % en todos los casos.

* **Variabilidad y escalabilidad**
  Las pequeñas variaciones (por ejemplo, 0.01916 s vs 0.01969 s) se deben al muestreo aleatorio y a la carga puntual del sistema, pero el patrón es claro: casi toda la ganancia se consigue en la primera pasada de `find`, y en llamadas sucesivas el árbol ya está plano.

En resumen, la compresión de caminos:

1. **Aplana los árboles desde la primera iteración**, reduciendo la profundidad efectiva a casi 1.
2. **Garantiza un tiempo de respuesta casi constante** en cada `find`.
3. **Ofrece un speed‑up de un 25-30 %** en tu benchmark, con un sobrecoste mínimo en las uniones.



### **Pregunta 5**

In [ ]:
# kdtree.py

import numpy as np
from collections import namedtuple

class KDNode:
    def __init__(self, point=None, axis=0, left=None, right=None):
        self.point = point
        self.axis = axis
        self.left = left
        self.right = right

class KDTree:
    def __init__(self, points: np.ndarray):
        """
        points: array de forma (n, d)
        Construye el KDTree recursivamente, eligiendo la mediana según el eje.
        """
        self.root = self._build(points, depth=0)
    
    def _build(self, points: np.ndarray, depth: int):
        if len(points) == 0:
            return None
        axis = depth % points.shape[1]
        idx = np.argsort(points[:, axis])
        median = len(idx) // 2
        node = KDNode(
            point=points[idx[median]],
            axis=axis,
            left=self._build(points[idx[:median]], depth + 1),
            right=self._build(points[idx[median+1:]], depth + 1)
        )
        return node
    
    def nearest(self, query: np.ndarray):
        """
        Busca el vecino más cercano exacto al vector query.
        Poda comparando la distancia al hiperplano con la mejor distancia.
        """
        Best = namedtuple("Best", ["point", "dist"])
        best = Best(point=None, dist=float('inf'))
        
        def recurse(node: KDNode):
            nonlocal best
            if node is None:
                return
            # Distancia al punto actual
            d = np.linalg.norm(query - node.point)
            if d < best.dist:
                best = best._replace(point=node.point, dist=d)
            axis = node.axis
            diff = query[axis] - node.point[axis]
            close, away = (node.left, node.right) if diff < 0 else (node.right, node.left)
            recurse(close)
            # Poda: si la proyección al hiperplano puede contener un punto más cercano
            if abs(diff) < best.dist:
                recurse(away)
        
        recurse(self.root)
        return best


In [ ]:
# sstree.py

import numpy as np
from collections import namedtuple

class SSNode:
    def __init__(self, points: np.ndarray):
        """
        points: array de forma (n, d)
        Cada nodo almacena su centroide y radio de la esfera contenedora.
        """
        self.points = points
        self.center = points.mean(axis=0)
        self.radius = np.max(np.linalg.norm(points - self.center, axis=1))
        self.left = None
        self.right = None

class SSTree:
    def __init__(self, points: np.ndarray, leaf_size: int = 10, epsilon: float = 0.1):
        """
        points: array de forma (n, d)
        leaf_size: tamaño de hoja para detallar búsqueda lineal
        epsilon: factor de error para poda aproximada
        """
        self.epsilon = epsilon
        self.leaf_size = leaf_size
        self.root = self._build(points)
    
    def _build(self, points: np.ndarray):
        node = SSNode(points)
        if len(points) <= self.leaf_size:
            return node
        # Partición por varianza máxima
        axis = np.argmax(points.var(axis=0))
        sorted_idx = np.argsort(points[:, axis])
        median = len(sorted_idx) // 2
        left_pts = points[sorted_idx[:median]]
        right_pts = points[sorted_idx[median:]]
        node.left = self._build(left_pts)
        node.right = self._build(right_pts)
        return node
    
    def nearest(self, query: np.ndarray):
        """
        Busca el vecino más cercano aproximado.
        Poda si la cota inferior > best.dist * (1 + ε).
        """
        Best = namedtuple("Best", ["point", "dist"])
        best = Best(point=None, dist=float('inf'))
        
        def recurse(node: SSNode):
            nonlocal best
            if node is None:
                return
            # Cota inferior de distancia al nodo
            d_cent = np.linalg.norm(query - node.center)
            if d_cent - node.radius > best.dist * (1 + self.epsilon):
                return
            # Si es hoja, búsqueda lineal
            if node.left is None and node.right is None:
                for p in node.points:
                    d = np.linalg.norm(query - p)
                    if d < best.dist:
                        best = best._replace(point=p, dist=d)
                return
            recurse(node.left)
            recurse(node.right)
        
        recurse(self.root)
        return best


In [ ]:
# benchmark.py

import numpy as np
import time
#from kdtree import KDTree
#from sstree import SSTree

def generate_data(n: int, k: int = 5, dim: int = 5) -> np.ndarray:
    """Genera n puntos en R^dim según mezcla de k gaussianas."""
    pts = []
    for _ in range(k):
        mu = np.random.uniform(-10, 10, size=(dim,))
        cov = np.eye(dim)
        size = n // k
        pts.append(np.random.multivariate_normal(mu, cov, size))
    return np.vstack(pts)

np.random.seed(0)
points = generate_data(10000)
queries = np.random.randn(1000, 5)

# Construcción KDTree
t0 = time.time()
kdt = KDTree(points)
t_kd_build = time.time() - t0

# Construcción SS+-Tree
t0 = time.time()
sst = SSTree(points)
t_ss_build = time.time() - t0

# Batch de consultas KDTree
t0 = time.time()
kd_results = [kdt.nearest(q) for q in queries]
t_kd_query = time.time() - t0

# Batch de consultas SS+-Tree
t0 = time.time()
ss_results = [sst.nearest(q) for q in queries]
t_ss_query = time.time() - t0

# Validación de precisión
eps = sst.epsilon
hits = sum(1 for (kd, ss) in zip(kd_results, ss_results) if ss.dist <= kd.dist * (1 + eps))
accuracy = hits / len(queries) * 100

# Resultados
print("Tiempos de construcción (s):")
print(f"  KDTree:    {t_kd_build:.4f}")
print(f"  SS+-Tree:  {t_ss_build:.4f}\n")

print("Tiempos de 1000 consultas (s):")
print(f"  KDTree:    {t_kd_query:.4f}")
print(f"  SS+-Tree:  {t_ss_query:.4f}\n")
print(f"Precisión (ε={eps}): {accuracy:.2f}% de consultas dentro del factor ε")


### **Problema 6**

In [ ]:
import random
import numpy as np

class KMeans:
    def __init__(self, k, max_iters=100, init='kmeans++'):
        self.k = k
        self.max_iters = max_iters
        self.init = init
        self.centroids = None

    def _initialize_centroids(self, X):
        n, _ = X.shape
        if self.init == 'random':
            # Selección aleatoria de k puntos como centroides iniciales
            indices = random.sample(range(n), self.k)
            return X[indices].copy()
        # k-means++ con manejo de casos degenerados
        centroids = []
        centroids.append(X[random.randrange(n)])
        for _ in range(1, self.k):
            # Distancia al centro más cercano
            D2 = np.min(np.square(X[:, None] - np.array(centroids)[None, :]).sum(axis=2), axis=1)
            total = D2.sum()
            if total == 0 or np.isnan(total):
                # Todos los puntos están a distancia cero => elegir aleatoriamente
                centroids.append(X[random.randrange(n)])
                continue
            probs = D2 / total
            i = np.random.choice(n, p=probs)
            centroids.append(X[i])
        return np.array(centroids)

    def fit(self, X):
        n, d = X.shape
        # Inicialización de centroides
        self.centroids = self._initialize_centroids(X)
        for it in range(self.max_iters):
            # Asignación de puntos al centroide más cercano
            labels = np.argmin(((X[:, None] - self.centroids[None, :])**2).sum(axis=2), axis=1)
            new_centroids = np.array([X[labels == j].mean(axis=0) if np.any(labels==j) else self.centroids[j]
                                      for j in range(self.k)])
            # Criterio de parada: convergencia de centroides
            if np.allclose(self.centroids, new_centroids):
                break
            self.centroids = new_centroids
        self.labels_ = labels

    def predict(self, X):
        # Asignación para nuevos puntos
        return np.argmin(((X[:, None] - self.centroids[None, :])**2).sum(axis=2), axis=1)

In [ ]:
import numpy as np
from heapq import heappush, heappop

class OPTICS:
    def __init__(self, eps, min_pts):
        self.eps = eps
        self.min_pts = min_pts

    def _region_query(self, X, i):
        # Vecindario de radio eps
        dist = np.linalg.norm(X - X[i], axis=1)
        return np.where(dist <= self.eps)[0]

    def fit(self, X):
        n = X.shape[0]
        self.reachability_ = np.full(n, np.inf)
        self.processed = np.zeros(n, dtype=bool)
        self.ordering = []

        for i in range(n):
            if not self.processed[i]:
                self._expand_cluster_order(X, i)
        # self.ordering contiene el orden óptimo y reachability

    def _expand_cluster_order(self, X, i):
        neighbors = self._region_query(X, i)
        self.processed[i] = True
        self.ordering.append(i)
        core_dist = self._core_distance(X, i, neighbors)
        if core_dist is None:
            return
        # Heap para candidatos
        seeds = []
        self._update(X, i, neighbors, seeds, core_dist)
        while seeds:
            dist, j = heappop(seeds)
            if self.processed[j]:
                continue
            neighbors_j = self._region_query(X, j)
            self.processed[j] = True
            self.ordering.append(j)
            core_dist_j = self._core_distance(X, j, neighbors_j)
            if core_dist_j is not None:
                self._update(X, j, neighbors_j, seeds, core_dist_j)

    def _core_distance(self, X, i, neighbors):
        if len(neighbors) < self.min_pts:
            return None
        # Distancia al MinPts-ésimo vecino ordenado
        dists = np.linalg.norm(X[neighbors] - X[i], axis=1)
        return np.sort(dists)[self.min_pts - 1]

    def _update(self, X, i, neighbors, seeds, core_dist):
        for j in neighbors:
            if self.processed[j]:
                continue
            new_reach = max(core_dist, np.linalg.norm(X[i] - X[j]))
            if self.reachability_[j] == np.inf:
                self.reachability_[j] = new_reach
                heappush(seeds, (self.reachability_[j], j))
            elif new_reach < self.reachability_[j]:
                self.reachability_[j] = new_reach
                heappush(seeds, (self.reachability_[j], j))

#### **Pruebas**

In [ ]:
import unittest
import numpy as np
#from kmeans import KMeans
#from optics import OPTICS

class TestClustering(unittest.TestCase):
    def test_kmeans_simple(self):
        # Dos blobs Gaussianos en 2D
        np.random.seed(0)
        X1 = np.random.randn(50,2) + np.array([5,5])
        X2 = np.random.randn(50,2) + np.array([-5,-5])
        X = np.vstack([X1, X2])
        km = KMeans(k=2, max_iters=100)
        km.fit(X)
        labels = km.labels_
        # Debe haber 2 agrupaciones aproximadas
        self.assertTrue(set(labels) == {0,1})

    def test_kmeans_edge_cases(self):
        X = np.zeros((5,3))
        km = KMeans(k=3)
        km.fit(X)
        # Todos los centroides iguales
        self.assertTrue(np.allclose(km.centroids, 0))

    def test_optics_ring(self):
        # Anillo interior y exterior en 2D
        t = np.linspace(0,2*np.pi,100)
        R1 = np.c_[np.cos(t), np.sin(t)]
        R2 = np.c_[2*np.cos(t), 2*np.sin(t)]
        X = np.vstack([R1, R2])
        opt = OPTICS(eps=0.3, min_pts=5)
        opt.fit(X)
        # Verificar ordenamiento de alcance
        self.assertEqual(len(opt.ordering), X.shape[0])

    def test_optics_noise(self):
        X = np.array([[0,0]]*10 + [[10,10]])
        opt = OPTICS(eps=0.5, min_pts=3)
        opt.fit(X)
        # El punto aislado queda con reachability infinita al final
        self.assertTrue(opt.reachability_[-1] == np.inf)

if __name__ == '__main__':
    # Ejecutar solo los tests definidos, ignorando argumentos externos
    unittest.main(argv=[''], exit=False)


* **K-means** es muy eficiente (*O(n k d I)*) y funciona bien cuando los clusters son convexos y de densidad similar, pero requiere fijar **k** a priori, es sensible a outliers y no detecta bien formas no esféricas.
* **OPTICS** explora densidades variables y detecta ruido sin necesidad de especificar el número de clusters, soporta formas arbitrarias y anida jerarquías de densidad, pero su coste $O(n \log n)$ con índice espacial o $O(n^2)$ naïve y la parametrización de $\epsilon$ y **MinPts** pueden hacerlo más lento y complejo de afinar.
* **Recomendación**: emplea K-means en datos grandes, homogéneos y de clusters bien definidos por puntos centrales; recurre a OPTICS cuando esperes densidades dispares, ruido o estructuras no convexas, y valoras obtener una ordenación de alcance para analizar jerarquías de densidad.
